In [ ]:
#r "nuget: MathNet.Numerics"

open System.Linq;
open System.Collections.Generic
open System.Collections;
open MathNet.Numerics.LinearAlgebra
open MathNet.Numerics.Distributions

## Domain

In [ ]:
type Goal = 
    | Max
    | Min

type SquaredExponentialKernelParameters = { LengthScale : double; Variance : double }
type DataPoint       = { X : double; Y : double }
type GaussianProcess = 
    { 
        Kernel                   : SquaredExponentialKernelParameters 
        DataPoints               : List<DataPoint>
        mutable CovarianceMatrix : Matrix<double>
    }

type AcquistionFunctionResult = { X : double; Y : double }

type EstimationResult =
    { 
        Mean       : double
        LowerBound : double
        UpperBound : double
        X          : double
    }

type GaussianModel =
    {
        GaussianProcess : GaussianProcess 
        Query           : double -> double
        Inputs          : List<double> 
    }

type ModelResult = 
    { 
        Input                    : List<DataPoint>
        AcquistionFunctionResult : List<AcquistionFunctionResult>
        EstimationResult         : List<EstimationResult>
    }

## Functions

### Kernel 

In [ ]:
// Squared Exponential Kernel.
let gaussianKernelCompute (kernel: SquaredExponentialKernelParameters) (left : double) (right : double) : double  = 
    if left = right then kernel.Variance
    else 
        let squareDistance : double = Math.Pow( left - right, 2 )
        kernel.Variance * Math.Exp( -squareDistance / ( kernel.LengthScale * kernel.LengthScale * 2. ))

### Estimation 

In [ ]:
// Estimation Method.
let estimateAtPoint (gaussianProcess : GaussianProcess) (x : double) : EstimationResult = 
    let kStar : double[] = 
        gaussianProcess.DataPoints
                       .Select(fun dp -> gaussianKernelCompute gaussianProcess.Kernel x dp.X)
                       .ToArray()

    let yTrain : double[] = 
        gaussianProcess.DataPoints
                       .Select(fun dp -> dp.Y)
                       .ToArray()

    let ks         : Vector<double> = Vector<double>.Build.Dense kStar
    let f          : Vector<double> = Vector<double>.Build.Dense yTrain
    let common     : Vector<double> = gaussianProcess.CovarianceMatrix.Inverse().Multiply(ks)
    let mu         : double         = common.DotProduct f
    let confidence : double         = Math.Abs(-common.DotProduct(ks) + (gaussianKernelCompute gaussianProcess.Kernel x x))
    let estimation = { Mean = mu; LowerBound = mu - confidence; UpperBound = mu + confidence; X = x }
    estimation

In [ ]:
// Based on each subsequent iteration, we compute the next query point by referring to the surrogate function 
// that produces the highest acquisition value.
let estimateAtRange (gaussianProcess : GaussianProcess) (X : List<double>) : List<EstimationResult> =
    let result = X.Select(fun x -> estimateAtPoint gaussianProcess x).ToList()
    result

### Acquisition

In [ ]:
// Acquisition Method.
let expectedImprovement (gaussianProcess : GaussianProcess) 
                        (estimationResult : EstimationResult) 
                        (goal : Goal) : AcquistionFunctionResult = 

    // TODO: Improve this logic by keeping score of the max / min based on the goal.
    let bestValue : double = 
        match goal with
        | Goal.Max -> gaussianProcess.DataPoints.Max(fun l -> l.Y)
        | Goal.Min -> gaussianProcess.DataPoints.Min(fun l -> l.Y)

    if gaussianProcess.DataPoints.Any(fun d -> d.X = estimationResult.X) then { X = estimationResult.X ; Y = 0. } 
    else
        let delta : double = estimationResult.Mean - bestValue
        let sigma : double = estimationResult.UpperBound - estimationResult.LowerBound
        let z     : double = delta / sigma
        let next  : double = delta * Normal.CDF(0., 1., z) + sigma * Normal.PDF(0., 1., z)

        { X = estimationResult.X ; Y = Math.Max(next, 0) }

### Model

In [ ]:
let addDataPoint (model : GaussianModel) (dataPoint : DataPoint) : unit =

    model.GaussianProcess.DataPoints.Add dataPoint

    let size : int = model.GaussianProcess.DataPoints.Count
    let mutable updatedCovariance : Matrix<double> = Matrix<double>.Build.Dense(size, size) 

    for rowIdx in 0..(model.GaussianProcess.CovarianceMatrix.RowCount - 1) do
        for columnIdx in 0..(model.GaussianProcess.CovarianceMatrix.ColumnCount - 1) do
            updatedCovariance[rowIdx, columnIdx] <- model.GaussianProcess.CovarianceMatrix.[rowIdx, columnIdx]

    for runnerIdx in 0..(size - 1) do
        let value : double = gaussianKernelCompute model.GaussianProcess.Kernel model.GaussianProcess.DataPoints.[runnerIdx].X dataPoint.X
        updatedCovariance[runnerIdx, size - 1] <- value
        updatedCovariance[size - 1, runnerIdx] <- value
    
    updatedCovariance[size - 1, size - 1] <- gaussianKernelCompute model.GaussianProcess.Kernel dataPoint.X dataPoint.X
    updatedCovariance.MapInplace(fun q -> Math.Round(q, 5))
    model.GaussianProcess.CovarianceMatrix <- updatedCovariance

let addDataPointViaProcess (gaussianProcess : GaussianProcess) (dataPoint : DataPoint) : unit =

    gaussianProcess.DataPoints.Add dataPoint

    let size : int = gaussianProcess.DataPoints.Count
    let mutable updatedCovariance : Matrix<double> = Matrix<double>.Build.Dense(size, size) 

    for rowIdx in 0..(gaussianProcess.CovarianceMatrix.RowCount - 1) do
        for columnIdx in 0..(gaussianProcess.CovarianceMatrix.ColumnCount - 1) do
            updatedCovariance[rowIdx, columnIdx] <- gaussianProcess.CovarianceMatrix.[rowIdx, columnIdx]

    for runnerIdx in 0..(size - 1) do
        let value : double = gaussianKernelCompute gaussianProcess.Kernel gaussianProcess.DataPoints.[runnerIdx].X dataPoint.X
        updatedCovariance[runnerIdx, size - 1] <- value
        updatedCovariance[size - 1, runnerIdx] <- value
    
    updatedCovariance[size - 1, size - 1] <- gaussianKernelCompute gaussianProcess.Kernel dataPoint.X dataPoint.X
    updatedCovariance.MapInplace(fun q -> Math.Round(q, 5))
    gaussianProcess.CovarianceMatrix <- updatedCovariance

In [ ]:
let createModel (gaussianProcess  : GaussianProcess) 
                (query            : double -> double) 
                (min              : double) 
                (max              : double)
                (resolution       : int) : GaussianModel = 

    // Random Uniform Initialization of Inputs.
    let inputs : List<double> = (seq { for i in 0 .. resolution do i }
                                |> Seq.map(fun idx -> min + double idx * (max - min) / (double resolution - 1.))).ToList()
    { GaussianProcess = gaussianProcess; Query = query; Inputs = inputs }

In [ ]:
let findExtrema (gaussianModel : GaussianModel) (goal : Goal) (iterations : int) : ModelResult = 
    addDataPointViaProcess gaussianModel.GaussianProcess { X = gaussianModel.Inputs.[0]; Y = gaussianModel.Query gaussianModel.Inputs.[0] }
    addDataPointViaProcess gaussianModel.GaussianProcess { X = gaussianModel.Inputs.Last(); Y = gaussianModel.Query ( gaussianModel.Inputs.Last() )}

    for iterationIdx in 0..(iterations - 1) do

        // Acquire next data point to explore.
        let nextPointToExplore : double = 
            // Find the data point that maximizes the acquisition function.
            let estimatedAtRange : List<EstimationResult> = estimateAtRange gaussianModel.GaussianProcess gaussianModel.Inputs
            let maxAcquisition   : List<AcquistionFunctionResult> = estimatedAtRange.Select(fun e -> (expectedImprovement gaussianModel.GaussianProcess e goal)).ToList()
            let maxVal = maxAcquisition.MaxBy(fun e -> e.Y)
            maxVal.X

        if gaussianModel.GaussianProcess.DataPoints.Any(fun d -> d.X = nextPointToExplore) then ()        
        else 
        addDataPoint gaussianModel { X = nextPointToExplore; Y = gaussianModel.Query ( nextPointToExplore )}

    let estimationResult : List<EstimationResult> = estimateAtRange gaussianModel.GaussianProcess gaussianModel.Inputs

    {
        Input                    = gaussianModel.GaussianProcess.DataPoints
        AcquistionFunctionResult = estimationResult.Select(fun e -> expectedImprovement gaussianModel.GaussianProcess e goal).ToList() 
        EstimationResult         = estimationResult 
    }

## Tests

### E2E Test

In [ ]:
open MathNet.Numerics

let test_model() : GaussianModel =
    let gaussianProcess : GaussianProcess = 
        { 
            Kernel           = { LengthScale = 0.1; Variance = 1 }
            DataPoints       = List<DataPoint>()
            CovarianceMatrix = Matrix<double>.Build.Dense(1, 1)
        }

    createModel gaussianProcess Trig.Sin -Math.PI Math.PI 300 

In [ ]:
open System
open System.Linq
open System.Collections.Generic
open MathNet.Numerics.LinearAlgebra
open MathNet.Numerics
open System.Diagnostics

[<Literal>]
let workload : string = @"C:\Users\mukun\source\repos\FSharpAdvent_2022\src\Workloads\SimpleWorkload_1\bin\Debug\net7.0\SimpleWorkload_1.exe"

let query(input : double) : double = 

    let p = new Process()
    p.StartInfo.FileName        <- workload
    p.StartInfo.UseShellExecute <- false
    p.StartInfo.Arguments       <- $"--input {input}"

    let stopWatch = Stopwatch()
    stopWatch.Start()

    p.Start()       |> ignore
    p.WaitForExit() |> ignore

    p.Dispose()

    stopWatch.Stop()

    double stopWatch.ElapsedMilliseconds / 1000.

let test_model_2nd() : GaussianModel =
    let gaussianProcess : GaussianProcess = 
        { 
            Kernel           = { LengthScale = 0.1; Variance = 1 }
            DataPoints       = List<DataPoint>()
            CovarianceMatrix = Matrix<double>.Build.Dense(1, 1)
        }

    createModel gaussianProcess query 0 10 900 

let model = test_model_2nd()
let extrema = findExtrema model Goal.Min 40 
extrema.Input.Min(fun e -> e.Y)

### Test Add Point

In [ ]:
let gaussianProcess : GaussianProcess = 
    { 
        Kernel           = { LengthScale = 1; Variance = 1 }
        DataPoints       = List<DataPoint>()
        CovarianceMatrix = Matrix<double>.Build.Dense(1, 1)
    }
addDataPointViaProcess gaussianProcess { X = 1.02; Y = 0.79 }
addDataPointViaProcess gaussianProcess { X = 1.99; Y = 0.94 }
addDataPointViaProcess gaussianProcess { X = 4.04; Y = 0.65 }

for i in 0..gaussianProcess.CovarianceMatrix.RowCount - 1 do
    for j in 0..gaussianProcess.CovarianceMatrix.ColumnCount - 1 do
        printfn "i: %A | j: %A = %A" i j gaussianProcess.CovarianceMatrix.[i, j]

gaussianProcess.CovarianceMatrix.ToArray()

### Estimate At Point Tests

In [ ]:
let gaussianProcess : GaussianProcess = 
    { 
        Kernel           = { LengthScale = 1; Variance = 1 }
        DataPoints       = List<DataPoint>()
        CovarianceMatrix = Matrix<double>.Build.Dense(1, 1)
    }

addDataPointViaProcess gaussianProcess { X = 1.02; Y = 0.79 }
addDataPointViaProcess gaussianProcess { X = 1.99; Y = 0.94 }
addDataPointViaProcess gaussianProcess { X = 4.04; Y = 0.65 }

estimateAtPoint gaussianProcess 3.

### Estimate At Range Tests

In [ ]:
let gaussianProcess : GaussianProcess = 
    { 
        Kernel           = { LengthScale = 1; Variance = 1 }
        DataPoints       = List<DataPoint>()
        CovarianceMatrix = Matrix<double>.Build.Dense(1, 1)
    }
addDataPointViaProcess gaussianProcess { X = 1.02; Y = 0.79 }
addDataPointViaProcess gaussianProcess { X = 1.99; Y = 0.94 }
addDataPointViaProcess gaussianProcess { X = 4.04; Y = 0.65 }

let list = List<double>()
list.Add(1.02)
list.Add(1.775)
list.Add(2.530)
list.Add(3.285)
list.Add(4.04)
estimateAtRange gaussianProcess list

In [ ]:
System.Diagnostics.Process.GetCurrentProcess().Id.Display()

#!about